# Scrapping para coletar os tickers das Ações, BDR's e FII's listadas

In [ ]:
from tqdm import tqdm
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
class Ticker_scrapper:
    
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
    
    def _get_html(self):
        try:
            self.req = Request(self.url, headers=self.headers)
            self.response = urlopen(self.req)
            self.html = self.response.read()

        except HTTPError as e:
            raise ValueError(e.status, e.reason)

        except URLError as e:
            raise ValueError(e.reason)
    
    def get_tickers_bdr(self):
        self._get_html()
        self.soup = BeautifulSoup(self.html, 'html.parser')
        self.tickers = []
        
        for self.item in self.soup.findAll('td', class_='String Column2'):
            for self.item2 in self.item.findAll('a'):
                self.tickers.append(self.item2.get_text())
        return self.tickers
                
    def get_tickers_acao(self):
        self.tickers = []
        self.lista_opcoes = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        for self.opcao in tqdm(self.lista_opcoes):

            try:
                self.req = Request((self.url + self.opcao), headers=self.headers)
                self.response = urlopen(self.req)
                self.html = self.response.read()

            except HTTPError as e:
                raise ValueError(e.status, e.reason)

            except URLError as e:
                raise ValueError(e.reason)

            self.soup = BeautifulSoup(self.html, 'html.parser')
            self.itens = self.soup.find_all('td', class_="String Column2 ColumnLast")
            for self.item in self.itens:
                self.tickers.append(self.item.get_text())
        return self.tickers

                    
    def get_tickers_fii(self):
        self._get_html()
        self.soup = BeautifulSoup(self.html, 'html.parser')
        self.group_tickers = []
        self.tickers = []
        
        for self.item in self.soup.findAll('span', class_='ticker'):
            self.tickers.append(self.item.get_text())
        return self.tickers


In [ ]:
# BDR
url = 'https://br.advfn.com/indice/bdrx'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
tickers = Ticker_scrapper(url, headers).get_tickers_bdr()
len(tickers)

In [ ]:
# Ação
url = 'https://br.advfn.com/bolsa-de-valores/bovespa/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
tickers = Ticker_scrapper(url, headers).get_tickers_acao()
len(tickers)

In [ ]:
# Fii's
url = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}
tickers = Ticker_scrapper(url, headers).get_tickers_fii()
len(tickers)

# Coletando as cotações das ações e links com o logo

In [ ]:
import pandas as pd
import yfinance as yf
pd.set_option('display.max_columns', None)

In [ ]:
class Historico_acoes:
    
    def __init__(self, tickers, start_date, get_img=1):
        self.tickers = tickers
        self.start_date = start_date
        self.index = yf.Ticker('^BVSP').history(start=self.start_date).index
        self.dados_historicos = pd.DataFrame(data=0,columns=[],index=self.index)
        self.rejected_volume = []
        self.rejected_ticker = []
        self.len_trouble = []
        self.imgs = {}
        self.get_img = get_img
    
    def get_close(self, volume_minimo):
        self.volume_minimo = volume_minimo
        for self.ticker_unico in tqdm(self.tickers):
            try:
                self.dados_historicos_ticker_unico = yf.Ticker(self.ticker_unico+'.SA').history(start=self.start_date)
                self._check_len()
                self._check_volume(self.volume_minimo)
                self._get_img()
                self._add_stock() if self.check_volume == True else self._rejected_volume()
            except:
                pass
    
    def _check_volume(self, volume_minimo):
        self.volume_minimo = volume_minimo
        self.volume_em_rs = (self.dados_historicos_ticker_unico['Volume'].mean() *
                             self.dados_historicos_ticker_unico['Close'].mean())
        self.check_volume = True if self.volume_em_rs > self.volume_minimo else False

    def _check_len(self):
        if len(self.dados_historicos_ticker_unico) > len(self.dados_historicos):
            self.diference = self.dados_historicos_ticker_unico.index.difference(self.dados_historicos.index)
            self.dados_historicos_ticker_unico.drop(self.diference, inplace=True)
            self._len_trouble()


    def _add_stock(self):
        self.dados_historicos[self.ticker_unico] = self.dados_historicos_ticker_unico['Close']
        
    def _len_trouble(self):
        self.len_trouble.append(self.ticker_unico)
        
    def _rejected_volume(self):
        self.rejected_volume.append(self.ticker_unico)
        
    def _reject_ticker(self):
        self.rejected_ticker.append(self.ticker_unico)
        
    def show_result(self):
        return self.dados_historicos
        
    def show_rejected_volume(self):
        return self.rejected_volume
    
    def show_len_trouble(self):
        return self.len_trouble
    
    def _get_img(self):
        if self.get_img == 1:
            try:
                self.url = 'https://www.google.com/search?q=' + str(self.ticker_unico)
                self.chrome_options = Options()
                self.chrome_options.add_argument("--headless")
                self.driver = webdriver.Chrome(chrome_options=self.chrome_options)
                self.driver.get(self.url)
                self.html = self.driver.page_source
                self.soup = BeautifulSoup(self.html,'html.parser')
                self.img = self.soup.find('g-img', class_='ivg-i')
                self.endereco_imagem = self.img.get('data-lpage')
                self.imgs[self.ticker_unico] = self.endereco_imagem
            except:
                self.imgs[self.ticker_unico] = 0

In [ ]:
start_date = '2020-03-23'  # Data do menor valor da ibovespa após o covid
historico = Historico_acoes(tickers, start_date)
historico.get_close(5000000)
historico.show_result()

###### Exportando para csv pois o processo acima é muito demorado.

In [ ]:
historico.show_result().to_csv('Historico_covid.csv', sep=';')

In [ ]:
imagens = historico.imgs
imagens2 = pd.DataFrame.from_dict(imagens, orient='index')

In [ ]:
imagens2.to_csv('imagens.csv', sep=';')

# Checando os dados

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
historico = pd.read_csv('Historico_bdr.csv', sep=';')

In [ ]:
# historico = historico[:-1]
historico

In [ ]:
# Colunas onde existe NaN
colunas_com_nan = []
for column in historico.columns:
    colunas_com_nan.append(column) if historico[column].isnull().values.any() else None
colunas_com_nan

In [ ]:
historico_nan = historico[colunas_com_nan]
historico.drop(colunas_com_nan, axis=1, inplace=True)

In [ ]:
historico.isnull().values.any()

In [ ]:
historico['Date'] = historico['Date'].astype('datetime64[ns]')
historico.set_index('Date', drop=True, inplace=True)
historico

In [ ]:
len(colunas_com_nan)

In [ ]:
historico_nan.shape

In [ ]:
for item in colunas_com_nan:
    print(item) if item not in historico_nan.columns else None

In [ ]:
historico_nan = historico_nan[:-1]
historico_nan

In [ ]:
# Os valores com NaN foram listadas apos 2018-01-01, logo serão preenchidas com o primeiro valor existente
# Isso faz com que a porcentagem fique 0 até a ação ser listada
historico_nan.fillna(method='bfill', inplace=True)
historico_nan

###### Unindo os dois DataFrames

In [ ]:
historico

In [ ]:
if len(historico_nan) > len(historico):
    diference = historico_nan.index.difference(historico.index)
    historico_nan.drop(diference, inplace=True)
historico = historico.join(historico_nan)
historico

# Preparando para construção do gráfico

In [ ]:
# Os dados serão convertidos para porcentagem
# Antes serão coletados apenas dados em um intervalo de 10 dias, para diminuir a volatilidade da movimentação do gráfico
historico_10_dias = historico.iloc[::10]
historico_10_dias

In [ ]:
historico_10_dias_porcentagem = pd.DataFrame(data=0, index=historico_10_dias.index, columns=[])
for column in historico_10_dias:
    historico_10_dias_porcentagem[column] = (historico_10_dias[column].divide(historico_10_dias[column][0]) - 1) * 100
historico_10_dias_porcentagem

In [ ]:
historico_10_dias_porcentagem.to_csv('Historico_10_dias_porcentagem_bdr.csv', sep=';', decimal=',')

In [ ]:
a = yf.Ticker('^BVSP').history(start='2020-01-01').Close
a.to_csv('bvsp.csv', sep=';', decimal=',')